In [1]:
from makiflow.save_recover import Builder
from makiflow.generators.ssd import LoadDataMethod
from makiflow.generators.ssd.gen_layers import InputGenLayerV2
from makiflow.generators.ssd.path_generators import RandomGeneratorSegment
import numpy as np
import makiflow as mf
mf.set_main_gpu(2)
from sklearn.utils import shuffle
from makiflow.metrics.od_metrics import mAP_maki_supported
import json
from glob import glob

In [2]:
map_method = LoadDataMethod(
    image_shape=[300, 300, 3],
    loc_shape=[2999, 4],
    loc_mask_shape=[2999],
    label_shape=[2999]
)

In [3]:

records = []
#fol = glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_num_3/*')

#for i in range(len(fol)):
#    if 'test' not in fol[i]:
#        records += glob(fol[i] + '/*.tfrecord')

# 2012 75k
records += glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/origin/*.tfrecord') 

records += glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_1_3_brigth/*.tfrecord')

records += glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/original_brigth_noizy/*.tfrecord')

records += glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_0_3_dark/*.tfrecord')

records += glob(f'/mnt/data/voc2012/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_2_brigth_noizy/*.tfrecord')

# 2007 20k
records += glob(f'/mnt/data/voc2007/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/origin/*.tfrecord')

records += glob(f'/mnt/data/voc2007/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_0_3_dark/*.tfrecord')

records += glob(f'/mnt/data/voc2007/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_2_dark_noizy/*.tfrecord')

records += glob(f'/mnt/data/voc2007/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/original_brigth_noizy/*.tfrecord')

records += glob(f'/mnt/data/voc2007/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_1_3_brigth/*.tfrecord')

# 2007 test 20k

records += glob(f'/mnt/data/voc_2007_test/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/origin/*.tfrecord')

records += glob(f'/mnt/data/voc_2007_test/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_2_dark_noizy/*.tfrecord')

records += glob(f'/mnt/data/voc_2007_test/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_1_3_brigth/*.tfrecord')

records += glob(f'/mnt/data/voc_2007_test/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/original_brigth_noizy/*.tfrecord')

records += glob(f'/mnt/data/voc_2007_test/tfr_sets/Mobilenetv2_1_data_rcnn_05iou/flip_0_3_dark/*.tfrecord')

In [4]:
len(records)

370

In [5]:
gen = RandomGeneratorSegment(records)

In [6]:
batch_size = 128

In [7]:
#in_x = InputGenLayer(2, batch_size, records, name='input', map_operation=map_method, num_parallel_calls=5)
in_x = InputGenLayerV2(
    6,
    batch_size,
    gen,             
    name='input',
    map_operation=map_method, 
    num_parallel_calls=10,
    cycle_length=7,
    block_length=160,
    shuffle=True, 
    buffer_size=128*7,
)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [8]:
model = Builder.ssd_from_json('Mobilenetv2_1_rcnn_style/mobilenetv2_noBias_eps1_3.json', generator=in_x)

Model is recovered.


In [9]:
in_x.get_iterator()

{'image': <tf.Tensor 'IteratorGetNext:0' shape=(128, 300, 300, 3) dtype=float32>,
 'loc': <tf.Tensor 'IteratorGetNext:2' shape=(128, 2999, 4) dtype=float32>,
 'loc_mask': <tf.Tensor 'IteratorGetNext:3' shape=(128, 2999) dtype=float32>,
 'label': <tf.Tensor 'IteratorGetNext:1' shape=(128, 2999) dtype=int32>}

In [10]:
import tensorflow as tf

In [11]:
#model.set_huber_loss()

In [12]:
model.set_generator(in_x)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
ses = tf.Session()

Create a change of learning rate

In [15]:
global_step = tf.Variable(0, trainable=False)

one_epoch = int((len(records) * 320) / batch_size)

def get_piecewise_constant_decay():
    global global_step, one_epoch
    boundaries = [one_epoch * 30]
    boundaries += [boundaries[-1] + one_epoch * 15]
    values = [1e-2, 1e-3, 1e-4]
    return tf.compat.v1.train.piecewise_constant(global_step, boundaries,
        values, name='supa_optimizer_lr')

def get_exp_decay(lr):
    global global_step, one_epoch
    return tf.compat.v1.train.exponential_decay(lr,
       global_step, one_epoch*2, 0.92, staircase=True, name='supa_optimizer_lr')

def get_cosine_decay(lr, steps=1,t_mul=2.0,m_mul=1.0,alpha=1e-4):
    global global_step, one_epoch
    return tf.train.cosine_decay_restarts(learning_rate=lr,
        global_step=global_step,
        first_decay_steps=one_epoch*steps,
        t_mul=t_mul,
        m_mul=m_mul,
        alpha=alpha,
        name='supa_optimizer_lr')

lr = get_piecewise_constant_decay()
ses.run(tf.initialize_variables([global_step]))

Instructions for updating:
Use `tf.variables_initializer` instead.


In [16]:
model.set_session(ses)

In [ ]:
reg

Set up test data

In [26]:
#from makiflow.models.ssd.tools import DataPreparator
from makiflow.models.ssd.tools.data_preparator_v2 import DataPreparatorV2 as DataPreparator

In [27]:
with open('data/classes.json', 'r') as f:
    name2class = json.load(f)

with open('data/test_valset_2012.json', 'r') as f:
    d = json.load(f)

preparator = DataPreparator(d, name2class, '/mnt/data/voc2012/VOCdevkit/VOC2012/JPEGImages/')
preparator.load_images()
preparator.resize_images_and_bboxes((300, 300))
imgs = preparator.get_images()

images = ((np.asarray(new_images) / 128) - 1).astype(np.float32)

true_boxes, true_labels = preparator.get_true_boxes_labels()

  0%|          | 5/2125 [00:00<00:49, 43.09it/s]

Loading images, bboxes and labels...


100%|██████████| 2125/2125 [00:31<00:00, 66.74it/s]
65it [00:00, 642.05it/s]

Images, bboxes and labels are loaded.


2125it [00:03, 650.29it/s]


In [30]:
len(imgs)

2825

In [31]:

def performe_test(model, name):
    preds = []
    for i in range(len(imgs) // batch_size):
        # preds = [confidences, locs]
        # `confidences` shape is [batch_sz, total_predictions, num_classes]
        # `locs` shape is [batch_sz, total_predictions, 4]
        preds += [model.predict(imgs[i*batch_size:(i+1)*batch_size])]
    
    p, r, ap, f1, unique_classes = mAP_maki_supported(preds, iou_threshold=0.5, conf_threshold=0.5, gboxes=true_boxes,glabels=true_labels)
    print('mAP is ',ap.mean())
    ap = np.round(ap,2)
    class2name = dict([(value,key) for key,value in name2class.items()])
    for i in range(len(ap)):
        print(class2name[i+1],'  ', ap[i])
    global_ap[name].append(ap.mean())

global_ap = {
    'focal': [],
    'maki': [],
    'scan': [],
    'topk': [],
    'quadratic': []
}

In [32]:
loss_info_top_k = {
    'positive losses': [],
    'negative losses': [],
    'total losses': [],
    'loc losses': []
}

loss_info_maki = {
    'Total loss':[],
    'Maki Loss':[],
    'Loc loss':[],
}

loss_info_focal = {
    'total loss':[],
    'focal loss':[],
    'loc loss':[],
}

loss_info_square = {
    'Total loss':[],
    'Maki Loss':[],
    'Loc loss':[],
}

In [34]:
epochs = 100
iterations = 925
loc_loss_w = 1.0 # 1.5
neg_ratio = 3.0 # 3.0
gamma = 2
# 8e-4
#opt = tf.train.AdamOptimizer(learning_rate=lr)
opt = tf.train.MomentumOptimizer(lr,momentum=0.9,use_nesterov=True)
#opt = tf.train.RMSPropOptimizer(learning_rate=lr, momentum=0.9, decay=0.999)

In [ ]:
# MAKI

In [ ]:
for i in range(epochs):
    info = model.genfit_maki(optimizer=opt, loc_loss_weight=loc_loss_w, gamma=gamma,
                                  epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'maki')
    loss_info_maki['Total loss'].append(info['Total loss'][0])
    loss_info_maki['Maki Loss'].append(info['Maki Loss'][0])
    loss_info_maki['Loc loss'].append(info['Loc loss'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'maki_{i}_zaet.ckpt')
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
# FOCAL

In [ ]:
for i in range(epochs):
    info = model.genfit_focal(optimizer=opt, loc_loss_weight=loc_loss_w, gamma=gamma,
                                  epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'focal')
    loss_info_focal['total loss'].append(info['total loss'][0])
    loss_info_focal['focal loss'].append(info['focal loss'][0])
    loss_info_focal['loc loss'].append(info['loc loss'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'focal_{i}_zaet.ckpt')
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
# SQUEARE

In [ ]:
for i in range(epochs):
    info = model.genfit_quadratic_ce(optimizer=opt, loc_loss_weight=loc_loss_w,
                                  epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'quadratic')
    loss_info_square['Total loss'].append(info['Total loss'][0])
    loss_info_square['Maki Loss'].append(info['Maki Loss'][0])
    loss_info_square['Loc loss'].append(info['Loc loss'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'square_{i}_zaet.ckpt')
        
#model.save_weights('top_k_map36.ckpt')
#reset()

In [ ]:
# TOP K

In [ ]:
for i in range(epochs):
    info = model.genfit_top_k(optimizer=opt, loc_loss_weight=loc_loss_w, neg_samples_ratio=neg_ratio,
                              epochs=1, iterations=iterations, global_step=global_step)
    performe_test(model, 'topk')
    loss_info_top_k['positive losses'].append(info['positive losses'][0])
    loss_info_top_k['negative losses'].append(info['negative losses'][0])
    loss_info_top_k['total losses'].append(info['total losses'][0])
    loss_info_top_k['loc losses'].append(info['loc losses'][0])
    print('epoch is ', i)
    if i != 0 and i % 4000 == 0:
        model.save_weights(f'top_k_{i}_zaet.ckpt')
    #if i != 0 and i % 10 == 0:
    #    loc_loss_w *= 0.92
    #if i != 0 and i % 10 == 0:
    #    neg_ratio *= 0.96
        
#model.save_weights('top_k_map36.ckpt')
#reset()

100%|██████████| 925/925 [04:00<00:00,  2.17it/s]


Epoch: 0 Loc loss: 0.12096264771462877 Positive loss 2.0305578430831392 Negative loss 0.6629310876509676 Total loss 3.2083492829056275


  0%|          | 0/925 [00:00<?, ?it/s]

mAP is  0.09870135665725324
aeroplane    0.3
bottle    0.0
motorbike    0.23
train    0.24
sheep    0.0
person    0.16
bicycle    0.08
boat    0.0
pottedplant    0.0
diningtable    0.1
bird    0.09
horse    0.05
chair    0.0
car    0.09
sofa    0.04
tvmonitor    0.0
cow    0.0
bus    0.19
cat    0.24
dog    0.17
epoch is  0


100%|██████████| 925/925 [03:55<00:00,  4.28it/s]


...

100%|██████████| 925/925 [03:36<00:00,  4.31it/s]


Epoch: 0 Loc loss: 0.05640809891794232 Positive loss 1.2751207815943049 Negative loss 0.5663046949230988 Total loss 2.245397262385892


  0%|          | 0/925 [00:00<?, ?it/s]

mAP is  0.2827867186150176
aeroplane    0.48
bottle    0.05
motorbike    0.45
train    0.53
sheep    0.1
person    0.31
bicycle    0.3
boat    0.1
pottedplant    0.08
diningtable    0.3
bird    0.26
horse    0.37
chair    0.05
car    0.23
sofa    0.29
tvmonitor    0.15
cow    0.18
bus    0.47
cat    0.55
dog    0.42
epoch is  12


 46%|████▌     | 423/925 [01:38<01:56,  4.29it/s]

In [ ]:
2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(global_ap['focal'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['positive losses'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['negative losses'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['total losses'])
plt.show()

In [ ]:
plt.plot(loss_info_top_k['loc losses'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(global_ap['focal'])

In [ ]:
plt.plot(global_ap['maki'])

In [ ]:
plt.plot(global_ap['topk'])

In [ ]:
plt.plot(global_ap['scan'])

In [ ]:
plt.plot(global_ap['quadratic'])

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
from makiflow.models.ssd.ssd_utils import nms, draw_bounding_boxes
import matplotlib.pyplot as plt
def test_picture(images, conf_trashhold=0.6):
    global model
    global num2name

    classes, locs = model.predict(images)
    i = 0
    for class_pred, loc_pred, image in zip(classes, locs, images):
        pred = nms(loc_pred, class_pred, conf_trashhold=conf_trashhold, iou_trashhold=0.5)
        print(pred,' number is ',i)
        input_dict = {
          'bboxes': pred[0],
          'classes': [num2name[cl] for cl in pred[1]]
        }
        image_with_bb = draw_bounding_boxes(image, input_dict)
        fig = plt.figure()
        fig.set_size_inches(16, 9)

        axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
        axes.imshow(image_with_bb)
        fig.savefig('results/'  +f'{i}.jpg')
        i += 1
        plt.close('all')

In [ ]:
import cv2 
import os
import numpy as np
arrs = []

path = '/mnt/data/voc2012/VOCdevkit/VOC2012/JPEGImages/'
img = None
taken = None
for elem in d['test_data']:
        path_image = path + elem['filename']
        testPic = cv2.imread(path_image)#NAME
        testPic = cv2.resize(testPic,(300,300))
        #testPic = cv2.cvtColor(testPic, cv2.COLOR_BGR2RGB)
        cal = (np.array(testPic) / 255).astype(np.float32)
        arrs.append(cal)
        img = testPic
        taken = elem
        break

In [ ]:
len(arrs)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
taken

In [ ]:
test_picture(arrs[:32], conf_trashhold=0.6)

In [ ]:
num2name = dict([(value, name) for name, value in class2name.items()])

In [ ]:
num2name